<a href="https://colab.research.google.com/github/DurgaPrasad-R/FML/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

# Define the CNN architecture using nn.Sequential
model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(32, 64, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(64 * 7 * 7, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# Initialize the weights using kaiming initialization
for module in model.modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight, mode='fan_in', nonlinearity='relu')

# Set up the optimizer and learning rate scheduler
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

# Set up the data loader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * data.size(0)
        train_correct += torch.sum(torch.argmax(output, dim=1) == target).item()

    # Compute the average train loss and accuracy for the epoch
    train_loss /= len(train_loader.dataset)
    train_accuracy = train_correct / len(train_loader.dataset)

    # Update the learning rate
    lr_scheduler.step()

    # Test the model
    model.eval()
    test_loss = 0.0
    test_correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = F.cross_entropy(output, target)
            test_loss += loss.item() * data.size(0)
            test_correct += torch.sum(torch.argmax(output, dim=1) == target).item()

        # Compute the average test loss and accuracy for the epoch
        test_loss /= len(test_loader.dataset)
        test_accuracy = test_correct / len(test_loader.dataset)

    # Print the training and test statistics for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch [1/10], Train Loss: 0.1827, Train Accuracy: 0.9477, Test Loss: 0.0492, Test Accuracy: 0.9842
Epoch [2/10], Train Loss: 0.0419, Train Accuracy: 0.9875, Test Loss: 0.0379, Test Accuracy: 0.9870
Epoch [3/10], Train Loss: 0.0294, Train Accuracy: 0.9910, Test Loss: 0.0354, Test Accuracy: 0.9877
Epoch [4/10], Train Loss: 0.0243, Train Accuracy: 0.9927, Test Loss: 0.0323, Test Accuracy: 0.9892
Epoch [5/10], Train Loss: 0.0219, Train Accuracy: 0.9938, Test Loss: 0.0310, Test Accuracy: 0.9890
Epoch [6/10], Train Loss: 0.0207, Train Accuracy: 0.9942, Test Loss: 0.0320, Test Accuracy: 0.9891
Epoch [7/10], Train Loss: 0.0201, Train Accuracy: 0.9944, Test Loss: 0.0320, Test Accuracy: 0.9888
Epoch [8/10], Train Loss: 0.0199, Train Accuracy: 0.9944, Test Loss: 0.0318, Test Accuracy: 0.9888
Epoch [9/10], Train Loss: 0.0197, Train Accuracy: 0.9946, Test Loss: 0.0316, Test Accuracy: 0.9888
Epoch [10/10], Train Loss: 0.0196, Train Accuracy: 0.9946, Test Loss: 0.0316, Test Accuracy: 0.9888


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
torch.save(model.state_dict(), 'mnist_cnn.pt')

In [7]:
import matplotlib.pyplot as plt

In [15]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# define the model architecture
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(7*7*64, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# load the saved state dict
state_dict = torch.load('mnist_cnn.pt', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)



# Load the image and apply transformations
image = Image.open('/content/drive/MyDrive/Datasets/7.jpg').convert('L')  # convert to grayscale

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
image = transform(image).unsqueeze(0)  # add batch dimension


# Make prediction
with torch.no_grad():
    output = model(image)

prediction = torch.argmax(output, dim=1).item()
print(f"The predicted digit is {prediction}.")


The predicted digit is 7.
